In [89]:
import time
import numpy as np 
import pandas as pd
import os,sys,argparse
import re

def ptm_cdr_wseq(seq,match_pattern):

    '''
    Cdrs
    AA such like "NG" "NS" "DG" on CDRs ;PTM means post translational modification.
    match_patten = r'N[GS]|DG'


    Wholeseq
    AA such like "N?S" "N?T" on the whole clone sequence.
    match_patten = r'N.[ST]'
    '''
    
    match = re.findall(match_pattern,seq)
    count = len(match)


    return count


def cal_ptms(line):
    
    cdr1counts = ptm_cdr_wseq(line[0],r'N[GS]|DG')
    cdr2counts = ptm_cdr_wseq(line[1],r'N[GS]|DG')
    cdr3counts = ptm_cdr_wseq(line[2],r'N[GS]|DG')
    #wseqcounts = ptm_cdr_wseq(line[3],r'N.[ST]')
    #counts = cdr1counts + cdr2counts + cdr3counts + wseqcounts
    return pd.Series({'Cdr1ptm':cdr1counts,'Cdr2ptm':cdr2counts,'Cdr3ptm':cdr3counts})
    #return  cdr1counts,cdr2counts,cdr3counts


def locC(df):
    '''
    There is supposed to be two CYS on a whole clone sequence. Locate the position of two CYS.
    '''
    nclist = (df == 'C').sum(axis=0)
    loclist = nclist.sort_values(ascending = False).iloc[:2].index.tolist()
    
    return loclist




def indexC(row,loclist):
    clist = row[row == 'C'].index.tolist()    
    A = set(loclist) | set(clist)
    B = set(loclist) & set(clist)
    nlist = map(str,A - B)
    locate = ','.join(nlist)
    #return len(A - B),locate
    return pd.Series({'Cptm':len(A - B), 'Cptm_locate':locate})


df_merge_clone = pd.DataFrame([['GFTFSRYS','ITSSSRYI','AARCPACGDCCVYFYW','771'],\
['NGNGCCG','ITTTTTNGSNGNGNGT','AATCITGSNGSECTW','589'],['NGNCCGT','ITTTTTNGSNGNGN\
GT','AATCITGWGNGSECTW','334'],['NGNCCNGT','INSTTTNGSNGNGNGT','AATCICGGSNGSECTW','234'\
]],columns = ['Cdr1','Cdr2','Cdr3','Counts'])

df_merge_clone['seqlen']=df_merge_clone['Cdr3'].map(lambda x : len(x))
df_clone_filter = df_merge_clone[df_merge_clone['seqlen']==df_merge_clone['seqlen'].max()]
df_clone_filter
df = df_clone_filter['Cdr3'].str.split('',expand=True)
loclist = locC(df)

df_clone_filter[['Cptm','Cptm_locate']] = df.apply(indexC,loclist = loclist,axis = 1)

df_clone_filter[['Cdr1ptm','Cdr2ptm','Cdr3ptm']] = df_clone_filter.apply(cal_ptms,axis=1)
df_clone_filter
#pd.merge(df_clone_filter,df_clone_filter.apply(cal_ptms,axis=1),left_index=True, right_index=True)


df_clone_filter['Allptms'] = df_clone_filter.Cptm + df_clone_filter.Cdr1ptm + df_clone_filter.Cdr2ptm + df_clone_filter.Cdr3ptm
df_clone_filter

In [94]:
df.apply(indexC,loclist = loclist,axis = 1)


0    [4, 10,11,14,7]
2              [0, ]
3             [1, 6]
dtype: object

In [ ]:
#df_clone_filter[['Cptm','Cptm_locate']] = df.apply(indexC,loclist = loclist,axis = 1)

df_clone_filter[['Cdr1ptm','Cdr2ptm','Cdr3ptm']] = df_clone_filter.apply(cal_ptms,axis=1)
df_clone_filter
#pd.merge(df_clone_filter,df_clone_filter.apply(cal_ptms,axis=1),left_index=True, right_index=True)


df_clone_filter['Allptms'] = df_clone_filter.Cptm + df_clone_filter.Cdr1ptm + df_clone_filter.Cdr2ptm + df_clone_filter.Cdr3ptm
df_clone_filter

In [64]:
df.apply(indexC,loclist = [4,14],axis = 1)
#df_clone_filter['Cptm'],df_clone_filter['Cptm_locate'] = df.apply(indexC,loclist = [4,14],axis = 1)
#df_all = df_clone_filter.merge(df.apply(indexC,loclist = [4,14],axis = 1),left_index=True, right_index=True)
df_clone_filter.apply(cal_ptms,axis=1)
pd.merge(df_clone_filter,df_clone_filter.apply(cal_ptms,axis=1),left_index=True, right_index=True)

,Cdr1,Cdr2,Cdr3,Counts,seqlen,Cdr1ptm,Cdr2ptm,Cdr3ptm
0,GFTFSRYS,ITSSSRYI,AARCPACGDCCVYFYW,771,16,0,0,0
2,NGNCCGT,ITTTTTNGSNGNGNGT,AATCITGWGNGSECTW,334,16,1,4,1
3,NGNCCNGT,INSTTTNGSNGNGNGT,AATCICGGSNGSECTW,234,16,2,5,1


In [56]:
df = df_clone_filter['Cdr3'].str.split('',expand=True).T
df['numC'] = (df == 'C').sum(axis=1)
df.numC.sort_values(ascending = False).iloc[:2].index.tolist()

[14, 4]

In [12]:
df = df_clone_filter['Cdr3'].str.split('',expand=True)
list = (df == 'C').sum(axis=0)
list.sort_values(ascending = False).iloc[:2].index.tolist()

[4, 14]

In [15]:
clist = df.loc[0][df.loc[0]=='C'].index.tolist()
clist,loclist

([4, 7, 10, 11], [4, 14])

In [16]:
df.apply(indexC,loclist = [4,14],axis = 1)

0    (4, 10,11,14,7)
2              (0, )
3             (1, 6)
dtype: object

In [18]:
df = df_clone_filter['Cdr3'].map(lambda x : list(x))
df

TypeError: 'Series' object is not callable

In [19]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,A,A,R,C,P,A,C,G,D,C,C,V,Y,F,Y,W,
2,,A,A,T,C,I,T,G,W,G,N,G,S,E,C,T,W,
3,,A,A,T,C,I,C,G,G,S,N,G,S,E,C,T,W,


In [21]:
pd.__version__

'0.23.4'